<a href="https://colab.research.google.com/github/cristina-ramos/class_NLP/blob/main/Assignments/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import Counter
from nltk.corpus import stopwords
from nltk.probability import FreqDist
import numpy as np
import math
import operator
from IPython.core.interactiveshell import InteractiveShell
import random
from nltk.tokenize import word_tokenize
from collections import defaultdict
import string
import argparse
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [ ]:
import nltk
import glob
import string
import pandas as pd
nltk.download('punkt')
nltk.download('stopwords')
!wget 'http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz'
#not sure if we have to use wget but my colab wouldn't untar my file otherwise
!tar -xf review_polarity.tar.gz

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
--2020-11-03 00:28:09--  http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 52.201.128.190
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|52.201.128.190|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3127238 (3.0M) [application/x-gzip]
Saving to: ‘review_polarity.tar.gz’

review_polarity.tar 100%[===================>]   2.98M  4.75MB/s    in 0.6s    

2020-11-03 00:28:10 (4.75 MB/s) - ‘review_polarity.tar.gz’ saved [3127238/3127238]



In [ ]:
neg_entries = glob.glob('/content/txt_sentoken/neg/*.txt')
pos_entries = glob.glob('/content/txt_sentoken/pos/*.txt')
#getting number of entries for each folder
print("The number of positive entries are: " + str(len(pos_entries)))
print("The number of negative entries are: " + str((len(neg_entries))))

The number of positive entries are: 1000
The number of negative entries are: 1000


In [ ]:
#Creating and empty list to append both positive and negative datasets:
totalwords = []

#Cleaning the data and removing punctuation and combining:
def merge_texts(directory):
  for fn in directory:
    if fn in pos_entries:
      with open(fn) as f:
        message = f.read().split()
        for w in message:
          if w not in string.punctuation:
            totalwords.append(w)
    elif fn in neg_entries:
      with open(fn) as f:
        message = f.read().split()
        for w in message:
          if w not in string.punctuation:
            totalwords.append(w)

merge_texts(pos_entries)
merge_texts(neg_entries)
print("The total number of words in the dataset is:", len(totalwords))

The total number of words in the dataset is: 1294462


In [ ]:
#Getting the top 100 words:
freq_count = Counter(totalwords).most_common()
top100 = []
for word_count in freq_count[0:100]:
  word = word_count[0]
  top100.append(word)
print("List of the top 100 words:", top100)

List of the top 100 words: ['the', 'a', 'and', 'of', 'to', 'is', 'in', 'that', 'it', 'as', 'with', 'for', 'his', 'this', 'film', 'but', 'he', 'i', 'on', 'are', 'by', 'be', 'an', 'not', 'one', 'movie', 'who', 'at', 'from', 'was', 'have', 'has', 'her', 'you', 'they', 'all', "it's", 'so', 'like', 'about', 'out', 'more', 'when', 'which', 'their', 'up', 'or', 'what', 'some', 'just', 'if', 'there', 'she', 'him', 'into', 'even', 'only', 'than', 'no', 'we', 'good', 'most', 'time', 'its', 'can', 'will', 'story', 'been', 'would', 'much', 'also', 'other', 'get', 'character', 'do', 'them', 'very', 'two', 'characters', 'first', 'after', 'see', 'because', 'way', 'well', 'make', 'any', 'does', 'really', 'had', 'too', 'while', 'films', 'how', 'little', 'life', 'where', 'plot', 'off', 'people']


In [ ]:
#finding # of stopwords in the top 100 using nltk's stopwords corpus
stop_words = set(stopwords.words('english'))
stop_count = 0
for word in top100:
  if word in stop_words:
    stop_count += 1
print("The number of stopwords in the top 100 words are:", str(stop_count))

The number of stopwords in the top 100 words are: 74


In [ ]:
#Most frequent 1000 words:
top_1000 = dict(freq_count[0:1000])
print("Here are the top 1000 words:", top_1000)

Here are the top 1000 words: {'the': 76178, 'a': 37934, 'and': 35343, 'of': 33970, 'to': 31759, 'is': 25163, 'in': 21591, 'that': 15105, 'it': 12289, 'as': 11338, 'with': 10778, 'for': 9913, 'his': 9566, 'this': 9560, 'film': 8849, 'but': 8576, 'he': 7607, 'i': 7492, 'on': 7241, 'are': 6943, 'by': 6218, 'be': 6082, 'an': 5735, 'not': 5519, 'one': 5514, 'movie': 5429, 'who': 5376, 'at': 4965, 'from': 4948, 'was': 4934, 'have': 4897, 'has': 4714, 'her': 4502, 'you': 4421, 'they': 4276, 'all': 4248, "it's": 3696, 'so': 3578, 'like': 3543, 'about': 3518, 'out': 3436, 'more': 3341, 'when': 3250, 'which': 3155, 'their': 3115, 'up': 3096, 'or': 3096, 'what': 3070, 'some': 2980, 'just': 2900, 'if': 2786, 'there': 2755, 'she': 2686, 'him': 2630, 'into': 2616, 'even': 2554, 'only': 2482, 'than': 2438, 'no': 2405, 'we': 2357, 'good': 2313, 'most': 2298, 'time': 2280, 'its': 2265, 'can': 2230, 'will': 2193, 'story': 2110, 'been': 2045, 'would': 2041, 'much': 2022, 'also': 1965, 'other': 1924, 'get

In [ ]:
#removing stopwords now
top1000 = FreqDist(totalwords).most_common(1000)
in_vocab = [word for word in totalwords if word not in stop_words]
print(top1000[:10])
print(in_vocab[:10])
print(len(in_vocab))
#okay now i need the most freq 1000 from vocab
fdist = FreqDist(in_vocab)
vocab = fdist.most_common(1000)
print(len(vocab))
vocab[:10]

[('the', 76178), ('a', 37934), ('and', 35343), ('of', 33970), ('to', 31759), ('is', 25163), ('in', 21591), ('that', 15105), ('it', 12289), ('as', 11338)]
['sam', 'matthew', 'broderick', 'astronomer', 'small', 'american', 'town', 'engaged', 'teacher', 'linda']
705486
1000


[('film', 8849),
 ('one', 5514),
 ('movie', 5429),
 ('like', 3543),
 ('even', 2554),
 ('good', 2313),
 ('time', 2280),
 ('story', 2110),
 ('would', 2041),
 ('much', 2022)]

In [ ]:
#total texts in both datasets
totaltexts = pos_entries + neg_entries
texts = []
for fn in totaltexts:
  if fn in pos_entries:
    cat = 1
  else:
    cat = 0
  with open(fn) as f:
    message = f.read().split()
    message = [w.rstrip(string.punctuation) for w in message]
    message = [w.lower() for w in message]
    message = [i for i in message if i]
    message = [word for word in message if word not in stop_words]
    texts.append((message, cat))

In [ ]:
#extracting features from the total words...so if a word is present the ouput should be (word, 1) if the word is not present it shoudl be (word, 0)...is that correct?
def get_features(document):
  feature = dict()
  for key, value in vocab:
    if key in document:
      feature[key] = 1
    else:
      feature[key] = 0
  return feature
doc_features = [(get_features(message), cat) for (message, cat) in texts]
len(doc_features)
# doc_features
#works just a little slow
#I know you said with defaultdict you don't need to initialize missing keys to zero however when i used defaultdict(int) and commented out the missing keys parts I got another sample sizeerror...so I'm sticking to this method for now

2000

In [ ]:
#splitting the total words into train and test set...is this correct?
from sklearn.model_selection import train_test_split
traindata, testdata = train_test_split(doc_features, test_size=0.2, random_state=1)

In [ ]:
#getting features
train = pd.DataFrame(traindata, columns=['Features', 'Labels'])
test = pd.DataFrame(testdata, columns=['Features', 'Labels'])

In [ ]:
train_feats = train['Features']
test_feats = test['Features']

train_labels = train['Labels']
test_labels = test['Labels']

In [ ]:
#vectorizing features
vector = DictVectorizer(sparse = False)
x_train = vector.fit_transform(train_feats.values)
x_test = vector.transform(test_feats)

y_train = train_labels.values
y_test = test_labels.values
len(y_test)
#not sure if that's the correct #

400

In [ ]:
#creating sigmoid function:
def sigmoid (x, w, b):
  x = np.array(x)
  w = np.array(w)
  p = 1 / (1 + math.exp(-(np.dot(w, x) + b)))
  return p

In [ ]:
#log regression model:
def log_regression(docs, labels):
  w = [0] * 1000
  b = 0
  L = 0.1
  it = 10
  pred = []
  scores = []
  for it in range(it):
    for (x,y) in zip(docs, labels):
      prob = sigmoid(x, w, b)
      d_w = (prob - y)*x
      d_b = prob - y
      w = w - L * d_w
      b = b - L * d_b
      threshold = 0.5
    for x in x_test:
      result = sigmoid(x, w, b)
      pred.append(1 if result >= threshold else 0)
    report = classification_report(y_test, pred)
    scores.append(report)
    pred = []
  return scores

In [ ]:
lr = log_regression(x_train, y_train)
lr
#yay finally

['              precision    recall  f1-score   support\n\n           0       0.80      0.82      0.81       205\n           1       0.81      0.78      0.79       195\n\n    accuracy                           0.80       400\n   macro avg       0.80      0.80      0.80       400\nweighted avg       0.80      0.80      0.80       400\n',
 '              precision    recall  f1-score   support\n\n           0       0.77      0.87      0.81       205\n           1       0.84      0.72      0.78       195\n\n    accuracy                           0.80       400\n   macro avg       0.80      0.80      0.80       400\nweighted avg       0.80      0.80      0.80       400\n',
 '              precision    recall  f1-score   support\n\n           0       0.75      0.90      0.82       205\n           1       0.87      0.69      0.77       195\n\n    accuracy                           0.80       400\n   macro avg       0.81      0.79      0.79       400\nweighted avg       0.81      0.80      0.

In [ ]:
# print(x_train.shape)
# print(y_train.shape)

In [ ]:
#general idea of the formula for minibatches:

# for i in range(iterations):
#   np.random.shuffle(data)
##not sure shuffling is actually needed
#   for batch in random_minibatches(data, batch_size=32):
#     grad = compute_gradient(batch, params)
#     params = params - learning_rate * grad

In [ ]:
def minibatches(x, y, batch):
  mini_batches = []
  for i in np.arange(0, x.shape[0], batch):
    x_mini = x[i:i + batch]
    y_mini = y[i:i + batch]
    mini_batches.append((x_mini, y_mini))
  return mini_batches

In [ ]:
mb = minibatches(x_train, y_train, batch=32)
mb

In [ ]:
minis = minibatches(x_train, y_train, batch=32)
minis[0][1]

In [ ]:
def log_regress(docs, labels):
  w = [0] * 1000
  b = 0
  L = 0.1
  it = 10
  pred = []
  scores = []
  for it in range(it):
    for (b_x, b_y) in minibatches(docs, labels, batch=32):
      b_weights = [0]
      b_bias = 0
      prob = (b_x.dot(np.array(w) - b))
      d_w = (prob - b_y)(b_x)
      d_b = (prob - b_y)
      b_weights += d_w
      b_bias += d_b
      avg = b_weights / 1000
      w = avg - L * d_w
      b = b_bias - L * d_b
    return w, b

In [ ]:
testing = log_regress(x_train, y_train)
testing

>These are all other (failed) attempts I tried for the minibatch:

In [ ]:
# def compute_cost(x, y, params):
#     num_samples = len(x_train)
#     cost_sum = 0.0
#     for x,y in zip(x_train, y_train):
#         y_hat = np.dot(params, np.array([1.0, x]))
#         cost_sum += (y_hat - y) ** 2
    
#     cost = cost_sum / (num_samples * 2.0)
    
#     return cost

In [ ]:
# def lin_reg_batch_gradient_descent(x, y, params, alpha, max_iter):
#     iteration = 0
#     num_samples = len(x_train)
#     cost = np.zeros(max_iter)
#     params_store = np.zeros([2, max_iter])
    
#     while iteration < max_iter:
#         cost[iteration] = compute_cost(x, y, params)
#         params_store[:, iteration] = params
        
#         print('--------------------------')
#         print(f'iteration: {iteration}')
#         print(f'cost: {cost[iteration]}')
        
#         for x,y in zip(x_train, y_train):
#             y_hat = np.dot(params, np.array([1.0, x]))
#             gradient = np.array([1.0, x]) * (y - y_hat)
#             params += alpha * gradient/num_samples
            
#         iteration += 1
    
#     return params, cost, params_store

# params_0 = np.array([20.0, 80.0])
# alpha_batch = 0.1
# lin_reg_batch_gradient_descent(x_train, y_train, params=params_0, alpha=alpha_batch, max_iter=32)

In [ ]:
# def next_batch(x, y, batch_size):
#   for i in np.arange(0, x.shape[0], batch_size):
#     yield (x[i:i + batch_size], y[i:i + batch_size])

In [ ]:
# ap = argparse.ArgumentParser()
# ap.add_argument("-e", "--epochs", type=float, default=10)
# ap.add_argument("-a", "--alpha", type=float, default=0.1)
# ap.add_argument("-b", "--batch-size", type=int, default=32)
# args = vars(ap.parse_args())

In [ ]:
# learn_rates = [0.0]
# num_iter = 10
# models = {}
# for i in learn_rates:
#   print("Learning rate is: ", i)
#   models[i] = [x_train, y_train, x_test, y_test]
#   print("------------------------------")
# for i in learn_rates:
#   plt.plot(np.squeeze(models[i]["costs"]), label= str(model[i]["learn_rate"]))

# plt.ylabel('cost')
# plt.xlabel('iterations')
# legend = plt.legend(loc='upper center', shadow=True)
# frame = legend.get_frame()
# frame.set_facecolor('0.90')
# plt.show()